### In this file we will load and evaluate a trained SBERT model

In [34]:

from sentence_transformers.evaluation import InformationRetrievalEvaluator
import logging
import requests
import csv
import gzip
from io import BytesIO
import tarfile
import os
import numpy as np
print(os.getcwd())
from sentence_transformer_dtu import SentenceTransformer, util
import numpy as np


/Users/andreasrygaard/Documents/Studie/MSC/Deep learning/train_sbert


In [5]:
# load our trained sbert model
print(os.getcwd())
os.chdir("/Users/andreasrygaard/Documents/Studie/MSC/Deep learning/train_sbert")

# Specify the path to the model
model_path = "siamese_model"

# Load the model
model = SentenceTransformer(model_path)



/Users/andreasrygaard/Documents/Studie/MSC/Deep learning/train_sbert


In [24]:
tupac=r"/Users/andreasrygaard/Documents/717_Tupac Shakur.txt"
# read the file and split into 
file = open(tupac, 'rt')
text = file.read()
file.close()
# split into sentences 
from nltk import sent_tokenize
sentences = sent_tokenize(text)
print(sentences)

['Tupac Amaru Shakur ( TOO-pahk shə-KOOR; born Lesane Parish Crooks; June 16, 1971 – September 13, 1996), also known by his stage names 2Pac and Makaveli, was an American rapper.', 'He is widely considered one of the most influential and successful rappers of all time.', 'Shakur is among the best-selling music artists, having sold more than 75 million records worldwide.', "Much of Shakur's music has been noted for addressing contemporary social issues that plagued inner cities, and he is considered by fans to be a symbol of activism against inequality.", 'Shakur was born in New York City to parents who were both political activists and Black Panther Party members.', 'Raised by his mother, Afeni Shakur, he relocated to Baltimore in 1984 and to the San Francisco Bay Area in 1988.', 'With the release of his debut album 2Pacalypse Now in 1991, he became a central figure in West Coast hip hop for his conscious rap lyrics.', 'Shakur achieved further critical and commercial success with his f

In [36]:


qlist=["What is the capital of france?", "I have crippling depression","My car is very fast","I like to eat apples","I am feling good","Paris","Italy","Bankok"]
sentences_wq=["what is the name of tupacs mother"]+sentences
ar=model.encode(sentences_wq,normalize_embeddings=True)

# calculate the euclidian distance between the two sentences
list_score= []
for tens in ar:
    list_score.append(np.linalg.norm(ar[0]-tens))

for i,score in enumerate(list_score):
    
    print(score,sentences_wq[0],sentences_wq[i])


print(ar[0].shape)

0.0 what is the name of tupacs mother what is the name of tupacs mother
0.61915845 what is the name of tupacs mother Tupac Amaru Shakur ( TOO-pahk shə-KOOR; born Lesane Parish Crooks; June 16, 1971 – September 13, 1996), also known by his stage names 2Pac and Makaveli, was an American rapper.
1.3110211 what is the name of tupacs mother He is widely considered one of the most influential and successful rappers of all time.
1.1082324 what is the name of tupacs mother Shakur is among the best-selling music artists, having sold more than 75 million records worldwide.
1.089973 what is the name of tupacs mother Much of Shakur's music has been noted for addressing contemporary social issues that plagued inner cities, and he is considered by fans to be a symbol of activism against inequality.
0.95935935 what is the name of tupacs mother Shakur was born in New York City to parents who were both political activists and Black Panther Party members.
1.1285436 what is the name of tupacs mother Rais

### Download test-queries, relevant passages, and corpus

In [38]:
#downoad the test queries
logging.info("Download evaluation queries")
# URL of the test query file
url = "https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz"
# Download the file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the downloaded file using gzip and read its content
    with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as file:
        # Create an empty dictionary to store data
        test_queries_dict = {}
        # Parse the TSV file
        tsv_reader = csv.reader(file, delimiter='\t')

        # Store data in the dictionary
        for row in tsv_reader:
            query_id, query_text = row[0], row[1]
            test_queries_dict[query_id] = query_text
    
    # Now data_dict contains the content of the file in a dictionary

    logging.info("Data downloaded and stored test queries in dictionary successfully.")
else:
    logging.info("Failed to download the file.")


#Load the Queries to Relevant passages mapping for the Information Retrieval evaluation
# URL of the file
url = "https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-passagetest2019-top1000.tsv.gz"

# Download the file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the downloaded file using gzip and read its content
    with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as file:
        # Create an empty dictionary to store data
        query_to_corpus = {}
        # Parse the TSV file
        tsv_reader = csv.reader(file, delimiter='\t')

        # Store data in the dictionary
        for row in tsv_reader:
            query_id, relevant_passage_id = row[0], row[1]
            if query_id in query_to_corpus:
                query_to_corpus[query_id].append(relevant_passage_id)
            else:
                query_to_corpus[query_id] = [relevant_passage_id]
            
    
    # Now  query_to_corpus contains all the relevant passages for each query
    logging.info("Query to relevant corpus mapping downloaded and stored in dictionary successfully.")
else:
    logging.info("Failed to download the file.")


# Download Corpus

### Now we read the MS Marco dataset
data_folder = 'msmarco-data'

#### Read the corpus files, that contain all the passages. Store them in the corpus dict
corpus = {}         #dict in the format: passage_id -> passage. Stores all existent passages
collection_filepath = os.path.join(data_folder, 'collection.tsv')
if not os.path.exists(collection_filepath):
    tar_filepath = os.path.join(data_folder, 'collection.tar.gz')
    if not os.path.exists(tar_filepath):
        logging.info("Download collection.tar.gz")
        util.http_get('https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz', tar_filepath)

    with tarfile.open(tar_filepath, "r:gz") as tar:
        tar.extractall(path=data_folder)

logging.info("Read corpus: collection.tsv")
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        pid = int(pid)
        corpus[pid] = passage




  0%|          | 0.00/1.04G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Evaluate the model

'''This evaluation function should be fed with a large corpus (the one used for training)
in addition we need a list of queries and a list of relevant documents for each query
this gives us a score in three different metrics: Mean Reciprocal Rank (MRR), Recall@k, and Normalized Discounted Cumulative Gain'''
evaluator = InformationRetrievalEvaluator(queries=test_queries_dict, corpus=corpus, relevant_docs=query_to_corpus)